In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from typing import List, Tuple, NoReturn, Any, Optional, Union
import pandas as pd
import numpy as np
import random

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from tqdm.auto import tqdm, trange
import os

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, HfArgumentParser

In [6]:
dataset = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [78]:
model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
df = pd.read_csv("/opt/ml/data/bm25/bm25_top_10_0.csv")
df.head()

,question,id,context_id,context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,[14489 6795 2269 20097 52322 20516 17086 325...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,[47823 47817 47824 47827 47820 47822 47815 478...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,[ 474 4628 461 4626 470 27415 35174 4...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,[34116 41747 31585 37068 32746 50052 40673 332...,이 불화는 17세기말에서 18세기 초 팔공산 일대에서 활발한 활동을 펼친 의균의 제...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,[43524 15814 55303 18908 21030 50041 42964 202...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


In [12]:
answers = df.answers.to_list()
question = df.question.to_list()
original_context = df.original_context.to_list()
top_k_passage = []
target = []

In [13]:
len(answers), len(question), len(original_context)

(3952, 3952, 3952)

In [14]:
for idx, passages in enumerate(df.context.tolist()):
    p_list = passages.split('▦')
    
    if original_context[idx] in p_list:
        top_k_passage.append(p_list)
        target_index = p_list.index(original_context[idx])
        target.append(target_index)
    else :
        p_list = [original_context[idx]] + p_list[:-1]
        top_k_passage.append(p_list)
        target.append(0)

In [21]:
len(top_k_passage[3545]), target[3545]

(10, 1)

In [45]:
i = random.randrange(3952)
i
print(question[i], answers[i])
print(original_context[i], '\n', '*'*150)
print(target[i], len(top_k_passage[i]))
print(top_k_passage[i][target[i]], '\n', '*'*150)


모투부가 1959년에 다시 방문한 곳은? {'answer_start': [427], 'text': ['브뤼셀']}
1956년 모부투는 "de Banzy"의 필명 아래 신문 기사들을 쓰기 시작하였다. 자신의 군사와 저널리즘 경력들을 통하여 그는 자유주의적 벨기에의 편집자와 상급 벨기에인 사관 마르리에르 대령 같은 권력적인 유럽의 후원자들을 찾았다. 그는 또한 킨샤사에서 새로운 아프리카의 엘리트들 중에 전망을 얻기도 하였다. 그의 단 하나의 문제는 자신을 이성적이나 적합한 도덕성 특성들을 결핍한 타락한 젊은 남자로 숙고한 가톨릭 교회와 였다. 모부투는 자신의 인생을 통하여 교회에 적대를 남겼다. 그는 자신의 부인과 가톨릭 전례의 결혼식을 올리는 것을 거부하였고, 그는 대통령으로서 전형적으로 반 임파 진영과 함께 결합하였다.\n\n1958년 모부투는 벨기에의 식민지 업적의 예들로서 브뤼셀 세계 박람회에서 전시된 콩고인들의 큰 대표단과 함께 브뤼셀로 갔다. 1959년 그는 브뤼셀을 재방문하여 식민지 선전국 "인포르콩고" (Inforcongo)에서 도제 살이를 지켰다. 이 게시는 그에게 브뤼셀 유학을 위한 기회들을 주었다. 1959년부터 1960년까지 정치적으로 야망의 콩고인들은 정치적 연락망들을 건설하는 데 바빴다.\n\n모부투는 브뤼셀에서 지속적으로 살며 갑자기 크게 나타난 초기 독립을 위한 번영으로서 벨기에에서 콩고인 국민주의자들과 연락을 만들고 있던 정보, 외교적과 재정적 이익들에 의하여 찾아졌다. 모부투는 이 시기 동안 재정가, CIA, 콩고인 학생들과 벨기에 안전군들 중에 연락들을 만들었다. 모부투는 다가오는 콩고를 위한 독립을 숙고한 브뤼셀에서 원탁 회의에 참석하였다. 그는 1957년에 시작된 파트리스 루뭄바와 우정을 개발하였고, 브뤼셀에서 MNC/L 사무소의 우두머리로 임명되었다. 그는 독립하기 3주 전에 콩고로 귀국하였다. 
 ***********************************************************************

In [79]:
p_seqs = tokenizer(
            top_k_passage[i],
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=False,
        )  # (top_k, 512)

q_seqs = tokenizer(
    question[i],
    padding="max_length",
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False ,
)  # (1, 512)

In [80]:
p_seqs.input_ids.shape

torch.Size([10, 512])

In [81]:
q_seqs.input_ids.shape

torch.Size([1, 512])

In [82]:
tokenizer.decode(p_seqs.input_ids[9])

'[CLS] 모부투는 위원들의 지원이 주로 젊은 대학 졸업생들이었던 위원 칼리지로 결성하면서 제1공화국 동안 잠시 다스렸다. 군인이자 정치적 인물 둘다인 모부투는 군사 독재자가 되는 데 관심이 없었다. 육군은 그의 주요 근거였으나 그는 자신의 복잡한 정치적 망상 조직을 지속하였고 정책을 만드는 정부에서 주요 참가자였다. 그는 1961년 8월 새로운 의회가 소집될 때까지 다스렸으며, 시릴 아둘라가 총리로서와 함께 새로운 정부가 형성되었다. 카사부부는 대통령으로서 남아있었다. 이 시기 동안 권력을 위하여 4개의 단체들 - 카사부부 아래 민간 정부, 앙투안 [UNK] 아래 북부 지방들, 카탕가에서 촘베와 알베르 칼론지에 의하여 지도된 카사이 주에서 분리 단체가 우열을 다투고 있었다. 촘베를 제외한 전부는 소란스러운 2년 동안 다스린 아둘라 아래 새로운 정부를 형성하는 데 가입하였다. 1964년 7월 촘베는 총리직을 취하고, " 국가 일치의 정부 " 로 알려진 새로운 국민 정부를 형성하는 데 촘베를 초청하였다. 8월 1일의 새 헌법은 연방주의의 조직을 합병한 대통령 제도를 설립하였다. = = 쿠데타를 이끌며 = = 제1공화국이었던 5년 완패 후, 모부투는 권력을 취하여 " 정치적 지도자들은 이 국가의 시민들의 평안을 위하여 아무 여김없이 투쟁하는 무효의 권력을 위하여 정착하였다. " 고 선언하였다. 1965년 쿠데타에 이어 모부투는 정부 그대로 제도적인 체제를 간직하였고, 새로운 공무원들과 함께 직위들을 채웠다. 그는 국가 원수가 되었고, 그의 부하 대령 ( 후에 장군 ) 레오나르 물람바는 총리로 임명되었다. 헌법적 정면의 뒤로 군인 직원은 유일한 효과적인 권한이 되었다. 모부투의 대통령직의 첫 5년은 그의 지배들과 직무로 들어가는 권력의 합동을 보았다. 가상 무정부 상태의 6년 후, 모부투는 자이르에 어떤 법률과 질서를 가져오는 데 성공하였다. 그가 국가 원수가 되었을 때 그는 " 5년 동안 국가 [SEP]'

In [11]:
file_name = "dksahjklvjxzckl \
asdkljklzxjckljvcxzk \
ksdajklxzcvjklvxcz\
fjklasdjkldscv\
dfsjkljxckzl"

In [12]:
file_name

'dksahjklvjxzckl asdkljklzxjckljvcxzk ksdajklxzcvjklvxczfjklasdjkldscvdfsjkljxckzl'